Using SimpleITK library to open, visualize 3D volumes (.nrrd), and display metadata (resolution)

Used tutiorials from https://insightsoftwareconsortium.github.io/SimpleITK-Notebooks/

In [8]:
import SimpleITK as sitk
import numpy as np
import os

In [9]:
# path to MICCAI_data
dataPath = "/Users/victor/github/deep-graphs/victor/MICCAI_data/CHB_train/CHB_train_Case01/"
print(dataPath)

/Users/victor/github/deep-graphs/victor/MICCAI_data/CHB_train/CHB_train_Case01/


In [10]:
# load volume
imageName = "CHB_train_Case01_T2.nhdr"
imagePath = dataPath + imageName

reader = sitk.ImageFileReader()
reader.SetImageIO("NrrdImageIO")
reader.SetFileName(imagePath)
image = reader.Execute();

In [14]:
# get size of image
image.GetSize()

(512, 512, 512)

Sub-sample and crop to 159×207×79 voxels and resolution 1×1×2 mm^3

In [17]:
sitk.ShrinkImageFilter?

Init signature: sitk.ShrinkImageFilter()
Docstring:     
Reduce the size of an image by an integer factor in each dimension.


ShrinkImageFilter reduces the size of an image by an integer factor in each dimension.
The algorithm implemented is a simple subsample. The output image size
in each dimension is given by:

outputSize[j] = max( std::floor(inputSize[j]/shrinkFactor[j]), 1 );

NOTE: The physical centers of the input and output will be the same.
Because of this, the Origin of the output may not be the same as the
Origin of the input. Since this filter produces an image which is a
different resolution, origin and with different pixel spacing than its
input image, it needs to override several of the methods defined in ProcessObject in order to properly manage the pipeline execution model. In
particular, this filter overrides
ProcessObject::GenerateInputRequestedRegion() and
ProcessObject::GenerateOutputInformation() .

This filter is implemented as a multithreaded filter. It provide

In [16]:
def myshow(img, title=None, margin=0.05, dpi=80):
    nda = sitk.GetArrayViewFromImage(img)
    spacing = img.GetSpacing()
        
    ysize = nda.shape[0]
    xsize = nda.shape[1]
      
    figsize = (1 + margin) * ysize / dpi, (1 + margin) * xsize / dpi

    fig = plt.figure(title, figsize=figsize, dpi=dpi)
    ax = fig.add_axes([margin, margin, 1 - 2*margin, 1 - 2*margin])
    
    extent = (0, xsize*spacing[1], 0, ysize*spacing[0])
    
    t = ax.imshow(nda,
            extent=extent,
            interpolation='hamming',
            cmap='gray',
            origin='lower')
    
    if(title):
        plt.title(title)
        

# create grid image
grid = sitk.GridSource(outputPixelType=sitk.sitkUInt16,
    size=(250, 250),
    sigma=(0.5, 0.5),
    gridSpacing=(5.0, 5.0),
    gridOffset=(0.0, 0.0),
    spacing=(0.2,0.2))
myshow(grid, 'Grid Input')

In [ ]:
# create sub-sample filter
subsample = sitk.ShrinkImageFilter()

# apply filter to image
grid_new = grid.Execute()